In [1]:
import numpy as np
import gradientDescent as gd
import scipy.sparse as sparse
import scipy.linalg as la

In [2]:
def quad_max_descent(f, x0, tol=1e-5, maxiter=100, Df=None, Q=None):
    '''
        Input:
            f - quadratic func to minimize
            x0 - initial point
            tol - tolerance
            maxiter - max num of iterations
            Df - Exact gradient func handle
            Q - Exact hess matrix
        Output:
            xf - final aproximation of x*
            iterations - number of iterations to reach soln
    '''
    
    xf = x0
    n = len(x0)
    iterations = 0
    c_1 = 0.1
    c_2 = 0.9
    
    grad_f = gd.grad(f, xf) if Df is None else Df(xf)
    Q = gd.hess(f, xf) if Q is None else Q
    
    
    while iterations < maxiter and np.linalg.norm(grad_f, np.inf) > tol:
        
        #Find descent direction
        d = - grad_f / np.linalg.norm(grad_f)
        
        #Direction coef.
        alfa = -0.99 * (np.dot(grad_f, d))/np.dot(d, np.dot(Q, d))
        
        #Next iteration
        xf = xf + alfa * d
        iterations += 1
        grad_f = gd.grad(f, xf) if Df is None else Df(xf)
    
    return xf, iterations

In [3]:
A = la.pascal(4)
b = -np.ones(4)
f = lambda x : 1/2 * np.dot(x, np.dot(A, x)) + np.dot(b, x) + 1
x0 = np.array([4, 4, 4, 4])

quad_max_descent(f, x0)

(array([  9.99928304e-01,   1.62683422e-04,  -1.36114515e-04,
          3.91474596e-05]), 85)

In [4]:
grad_f = lambda x : np.dot(A, x) + b
hess_f = lambda x : A

quad_max_descent(f, x0, Df=grad_f, Q=hess_f(x0))

(array([  9.99915062e-01,   1.96182219e-04,  -1.62694973e-04,
          4.63100974e-05]), 87)

In [5]:
def max_descent_opt(f, x0, tol=1e-5, maxiter=100):
    '''
        Input:
            f - func to minimize
            x0 - initial point
            tol - tolerance
            maxiter - max num of iterations
        Output:
            xf - final aproximation of x*
            iterations - number of iterations to reach soln
    '''
    
    xf = x0
    n = len(x0)
    iterations = 0
    c_1 = 0.1
    c_2 = 0.9
    
    grad_f = gd.grad(f, xf)
    
    while iterations < maxiter and np.linalg.norm(grad_f, np.inf) > tol:
        
        #Find descent direction
        d = - grad_f / np.linalg.norm(grad_f)
        
        #Direction coef.
        alfa = 1
        
        a = f(xf)
        b = np.dot(grad_f, d)
        
        while f(xf + alfa*d) > a + alfa*c_1*b:
            alfa /= 2
        
        alfa = 0.99*(-0.5*b*(alfa**2)/(f(xf+alfa*d) - a - b*alfa))
        assert np.dot(gd.grad(f, xf + alfa*d), d) >= c_2*np.dot(grad_f, d), "No cumple W2"
        
        #Next iteration
        xf = xf + alfa*d
        iterations += 1
        grad_f = gd.grad(f, xf)
    
    return xf, iterations

In [6]:
rosenbrock = lambda x : 100*(x[0]**2 - x[1])**2 + (x[0]-1)**2
x0_r = np.array([2, 3])
max_descent_opt(rosenbrock, x0_r, 1e-5, 1000)

(array([ 1.00000029,  1.00000061]), 446)

In [13]:
A = la.pascal(4)
b = -np.ones(4)
f = lambda x : 1/2 * np.dot(x, np.dot(A, x)) + np.dot(b, x) + 1
x0 = np.array([4, 4, 4, 4])

gd.trust_region(f, x0, 1, max_r=8)

(array([  9.99933224e-01,   1.57616529e-04,  -1.30026403e-04,
          3.69754826e-05]), 179)